In [33]:
import geopandas as gpd
from shapely.geometry import box
from shapely.ops import unary_union
from osgeo import gdal
from copy import deepcopy
import os, math
import numpy as np
import requests
import pandas as pd
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import zipfile
import warnings

In [34]:
os.chdir("C:/Users/ideapad Blue TDG/Desktop/GEOLOGIA/PRUEBAS") #CAMBIAR

In [35]:
#Buscar clave del municipio 
LIMITE_MUNICIPAL = gpd.read_file("//Mycloudex2ultra/compartido/Base_Datos/INEGI/MGN/MGN_2023/mg_2023_integrado/conjunto_de_datos/00mun.shp").to_crs(epsg=6372)
print ("Carga completa")

# Buscar un municipio por CLAVE DE ESTADO y NOMGEO
def buscar_municipio(nom_ent, nomgeo):
    resultado = LIMITE_MUNICIPAL[(LIMITE_MUNICIPAL['NOM_ENT'] == nom_ent) & (LIMITE_MUNICIPAL['NOMGEO'] == nomgeo)]    
    if not resultado.empty:
        resultado = resultado[['CVE_ENT', 'NOM_ENT', 'NOMGEO', 'CVEGEO']]
        print(f"Municipio encontrado:\n{resultado}")
    else:
        print(f"No se encontró el municipio con ESTADO '{nom_ent}' y MUNICIPIO '{nomgeo}'.")
    
    return resultado

Carga completa


In [122]:
 # CAMBIAR POR LA CLAVE DEL ESTADO
nom_ent = 'Baja California Sur'#CAMBIAR POR EL NOMBRE DEL ESTADO
nomgeo = 'La Paz' #CAMBIAR POR EL NOMBRE DEL MUNICIPIO
resultado = buscar_municipio(nom_ent, nomgeo)

Municipio encontrado:
   CVE_ENT              NOM_ENT  NOMGEO CVEGEO
20      03  Baja California Sur  La Paz  03003


In [123]:
LIMITE_MUNICIPAL = gpd.read_file('//Mycloudex2ultra/compartido/Base_Datos/INEGI/MGN/MGN_2023/mg_2023_integrado/conjunto_de_datos/00mun.shp').to_crs(epsg=6372)
cvegeo = '03003' #CAMBIAR
cve_ent = '03' #CAMBIAR
municipio = LIMITE_MUNICIPAL[LIMITE_MUNICIPAL['CVEGEO'] == cvegeo]
zona_corte = gpd.GeoDataFrame(geometry=municipio.geometry, crs=LIMITE_MUNICIPAL.crs).to_crs(epsg=6372)  # SE PUEDE CAMBIAR POR CUALQUIER SHP PREVIAMENTE DEFINIDO
print(zona_corte)

                                             geometry
20  MULTIPOLYGON (((1677351.228 1380621.554, 16773...


In [124]:
zona_corte.to_file('buffer.shp') 

In [125]:
cartas = gpd.read_file('//Mycloudex2ultra/compartido/Base_Datos/INEGI/LIMITES DE CARTAS TOPOGRÁFICAS/250k/250k/indi250kcw_1.shp').to_crs(epsg=6372)
claves = cartas['CLAVE'].unique()
claves_en_zona_corte = gpd.sjoin(cartas, zona_corte, how='inner', predicate='intersects')
claves_cartas = claves_en_zona_corte['CLAVE'].unique()
print(claves_cartas)

['G12-8' 'G12-11' 'G12-12' 'F12-2' 'F12-3']


In [126]:
# Ruta al WebDriver de Chrome
webdriver_path = r'C:\Users\ideapad Blue TDG\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('headless=False')
options.add_argument('--disable-gpu')
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service, options=options)

In [127]:
url = 'https://www.inegi.org.mx/temas/geologia/#descargas'
driver.get(url)

In [128]:
wait = WebDriverWait(driver, 100)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input#bcm_resultBusqueda2_input_search')))

clave_enlace_list = []

def buscar_clave(clave):
    search_box = driver.find_element(By.CSS_SELECTOR, 'input#bcm_resultBusqueda2_input_search')
    search_button = driver.find_element(By.CSS_SELECTOR, 'button#bcm_buscar_btn_resultBusqueda2')
    driver.execute_script("arguments[0].scrollIntoView();", search_box)
    driver.execute_script("arguments[0].scrollIntoView();", search_button)
    search_box.clear()
    search_box.send_keys(clave)
    WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#bcm_buscar_btn_resultBusqueda2')))
    driver.execute_script("arguments[0].click();", search_button)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody')))

def procesar_filas(clave_buscar):
    rows = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')
    for row in rows:
        try:
            clave_element = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1) .tablesaw-cell-content')
            clave = clave_element.text.strip()
            serie_element = row.find_element(By.CSS_SELECTOR, 'td:nth-child(4) .tablesaw-cell-content')
            serie = serie_element.text.strip()
            enlace_descarga = None
            for i in [8, 9, 10]:
                try:
                    tipo_archivo_element = row.find_element(By.CSS_SELECTOR, f'td:nth-child({i}) img[src="/img/ico/ico_shp.png"]')
                    if tipo_archivo_element:
                        enlace_element = row.find_element(By.CSS_SELECTOR, f'td:nth-child({i}) a[href$=".zip"]')
                        enlace_descarga = enlace_element.get_attribute('href')
                        break
                except Exception:
                    continue
            if enlace_descarga:
                clave_enlace_list.append((clave, serie, enlace_descarga))
        
        except Exception as e:
            print(f"Error procesando fila: {e}")
            continue

for clave_buscar in claves_cartas:
    buscar_clave(clave_buscar)
    time.sleep(10)  # Esperar para que se carguen los resultados
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        procesar_filas(clave_buscar)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

if clave_enlace_list:
    for clave, serie, enlace in clave_enlace_list:
        print(f"Clave: {clave} - Serie: {serie} - Enlace de descarga: {enlace}")
else:
    print("No se encontraron resultados para las claves buscadas")

driver.quit()

Clave: G12-6 - Serie: I - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/tematicas/Geologia_hist/1_250_000/702825675134_s.zip
Clave: G12-12 - Serie: I - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/tematicas/Geologia_hist/1_250_000/702825675080_s.zip
Clave: G12-11 - Serie: I - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/tematicas/Geologia_hist/1_250_000/702825236748_s.zip
Clave: G12-10 - Serie: I - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/tematicas/Geologia_hist/1_250_000/702825675073_s.zip
Clave: G12-5 - Serie: I - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/tematicas/Geologia_hist/1_250_000/702825

In [129]:
def descargar_archivo(url, carpeta_destino, nombre_archivo):
    respuesta = requests.get(url, stream=True)
    ruta_archivo = os.path.join(carpeta_destino, nombre_archivo)
    
    with open(ruta_archivo, 'wb') as archivo:
        for chunk in respuesta.iter_content(chunk_size=32768):
            if chunk:
                archivo.write(chunk)
    
    print(f"Descargado: {ruta_archivo}")

serie_deseada = "I"  # SE PUEDE CAMBIAR POR LA SERIE O AÑO DE INTERÉS
carpeta_descargas = "CARTAS GEOLOGICAS"  # CAMBIAR
os.makedirs(carpeta_descargas, exist_ok=True)

for clave, serie, enlace in clave_enlace_list:
    if clave in claves_cartas and (serie_deseada == "" or serie == serie_deseada):
        nombre_archivo = f"{clave}.zip"
        descargar_archivo(enlace, carpeta_descargas, nombre_archivo)

print("Descarga completa.")

Descargado: CARTAS GEOLOGICAS\G12-12.zip
Descargado: CARTAS GEOLOGICAS\G12-11.zip
Descargado: CARTAS GEOLOGICAS\G12-8.zip
Descargado: CARTAS GEOLOGICAS\G12-12.zip
Descargado: CARTAS GEOLOGICAS\F12-3.zip
Descargado: CARTAS GEOLOGICAS\F12-2.zip
Descargado: CARTAS GEOLOGICAS\F12-3.zip
Descargado: CARTAS GEOLOGICAS\F12-2.zip
Descarga completa.


In [130]:
# Descomprimir los archivos ZIP en la carpeta principal
for archivo_zip in os.listdir(carpeta_descargas):
    if archivo_zip.endswith('.zip'):
        ruta_zip = os.path.join(carpeta_descargas, archivo_zip)
        nombre = os.path.splitext(archivo_zip)[0]
        carpeta = os.path.join(carpeta_descargas, nombre)
        os.makedirs(carpeta, exist_ok=True)
        with zipfile.ZipFile(ruta_zip, 'r') as archivo_zip_obj:
            archivo_zip_obj.extractall(carpeta)
            print(f"Descomprimido: {ruta_zip} en {carpeta}")

# Buscar y descomprimir ZIP dentro de 'conjunto_de_datos' en cada carpeta descomprimida
for carpeta_nombre in os.listdir(carpeta_descargas):
    ruta_carpeta = os.path.join(carpeta_descargas, carpeta_nombre)
    ruta_conjunto_datos = os.path.join(ruta_carpeta, 'conjunto_de_datos')
    
    if os.path.isdir(ruta_conjunto_datos):
        for archivo_zip in os.listdir(ruta_conjunto_datos):
            if archivo_zip.endswith('.zip'):
                ruta_zip = os.path.join(ruta_conjunto_datos, archivo_zip)
                nombre = os.path.splitext(archivo_zip)[0]
                carpeta = os.path.join(ruta_conjunto_datos, nombre)
                os.makedirs(carpeta, exist_ok=True)
                with zipfile.ZipFile(ruta_zip, 'r') as archivo_zip_obj:
                    archivo_zip_obj.extractall(carpeta)
                    print(f"Descomprimido: {ruta_zip} en {carpeta}")

print("Descompresión completa de todos los archivos ZIP dentro de 'conjunto_de_datos'.")

Descomprimido: CARTAS GEOLOGICAS\F12-2.zip en CARTAS GEOLOGICAS\F12-2
Descomprimido: CARTAS GEOLOGICAS\F12-3.zip en CARTAS GEOLOGICAS\F12-3
Descomprimido: CARTAS GEOLOGICAS\G12-11.zip en CARTAS GEOLOGICAS\G12-11
Descomprimido: CARTAS GEOLOGICAS\G12-12.zip en CARTAS GEOLOGICAS\G12-12
Descomprimido: CARTAS GEOLOGICAS\G12-8.zip en CARTAS GEOLOGICAS\G12-8
Descomprimido: CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1.zip en CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1
Descomprimido: CARTAS GEOLOGICAS\F12-3\conjunto_de_datos\cdgv_1_250_000_serie1.zip en CARTAS GEOLOGICAS\F12-3\conjunto_de_datos\cdgv_1_250_000_serie1
Descomprimido: CARTAS GEOLOGICAS\G12-11\conjunto_de_datos\cdgv_1_250_000_serie1.zip en CARTAS GEOLOGICAS\G12-11\conjunto_de_datos\cdgv_1_250_000_serie1
Descomprimido: CARTAS GEOLOGICAS\G12-12\conjunto_de_datos\cdgv_1_250_000_serie1.zip en CARTAS GEOLOGICAS\G12-12\conjunto_de_datos\cdgv_1_250_000_serie1
Descomprimido: CARTAS GEOLOGICAS\G12-8\con

In [131]:
carpeta_zip = 'CARTAS GEOLOGICAS'

def buscar_archivos_shp(carpeta):
    """Busca archivos .shp en la carpeta dada."""
    archivos_shp = []
    for root, dirs, files in os.walk(carpeta):
        archivos_shp.extend([os.path.join(root, archivo) for archivo in files if archivo.endswith('.shp')])
    return archivos_shp

# Buscar archivos .shp en las subcarpetas descomprimidas
for carpeta in os.listdir(carpeta_zip):
    ruta_carpeta = os.path.join(carpeta_zip, carpeta)
    if os.path.isdir(ruta_carpeta):
        # Busca en la carpeta 'conjunto_de_datos'
        ruta_conjunto_datos = os.path.join(ruta_carpeta, 'conjunto_de_datos')
        if os.path.isdir(ruta_conjunto_datos):
            # Buscar archivos .shp en 'conjunto_de_datos'
            archivos_shp = buscar_archivos_shp(ruta_conjunto_datos)
            if archivos_shp:
                print(f"Archivos .shp encontrados en {ruta_conjunto_datos}:")
                for archivo in archivos_shp:
                    print(f"  - {archivo}")
            else:
                print(f"No se encontraron archivos .shp en {ruta_conjunto_datos}.")
                
            # Buscar archivos .shp en 'cdgv_1_250_000_serie1' dentro de 'conjunto_de_datos'
            ruta_cdgv_1_250_000_serie1 = os.path.join(ruta_conjunto_datos, 'cdgv_1_250_000_serie1')
            if os.path.isdir(ruta_cdgv_1_250_000_serie1):
                archivos_shp = buscar_archivos_shp(ruta_cdgv_1_250_000_serie1)
                if archivos_shp:
                    print(f"Archivos .shp encontrados en {ruta_cdgv_1_250_000_serie1}:")
                    for archivo in archivos_shp:
                        print(f"  - {archivo}")
                else:
                    print(f"No se encontraron archivos .shp en {ruta_cdgv_1_250_000_serie1}.")
                
print("Búsqueda de archivos .shp completa.")

Archivos .shp encontrados en CARTAS GEOLOGICAS\F12-2\conjunto_de_datos:
  - CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202lg.shp
  - CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202pg.shp
  - CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202ptg.shp
Archivos .shp encontrados en CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1:
  - CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202lg.shp
  - CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202pg.shp
  - CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202ptg.shp
Archivos .shp encontrados en CARTAS GEOLOGICAS\F12-3\conjunto_de_datos:
  - CARTAS GEOLOGICAS\F12-3\conjunto_de_datos\cdgv_1_250_000_serie1\f1203lg.shp
  - CARTAS GEOLOGICAS\F12-3\conjunto_de_datos\cdgv_1_250_000_serie1\f1203pg.shp
  - CARTAS GEOLOGICAS\F12-3\conjunto_de_datos\cdgv_1_250_000_serie1\f1203ptg.shp
Archivos .shp encontrados en CARTAS GEO

In [133]:
crs_original = "EPSG:26712"
crs_final = "EPSG:6372"

def buscar_archivos_shp(carpeta):
    """Busca archivos .shp en la carpeta dada y todas sus subcarpetas."""
    archivos_shp = []
    for root, dirs, files in os.walk(carpeta):
        archivos_shp.extend([os.path.join(root, archivo) for archivo in files if archivo.endswith('.shp')])
    return archivos_shp

def asignar_sistema_coordenadas(gdf, crs):
    """Asigna un sistema de coordenadas a un GeoDataFrame."""
    gdf = gdf.set_crs(crs, allow_override=True)
    return gdf

def reproyectar_sistema_coordenadas(gdf, crs):
    """Reproyecta un GeoDataFrame a un sistema de coordenadas diferente."""
    return gdf.to_crs(crs)

# Recopilar archivos por terminación
grupos_shp = {'lg': [], 'pg': [], 'ptg': []}

# Carpeta base
carpeta_zip = 'CARTAS GEOLOGICAS'

# Recorrer todas las subcarpetas en 'CARTAS GEOLOGICAS'
for carpeta in os.listdir(carpeta_zip):
    ruta_carpeta = os.path.join(carpeta_zip, carpeta)
    if os.path.isdir(ruta_carpeta):
        # Buscar en la carpeta 'conjunto_de_datos'
        ruta_conjunto_datos = os.path.join(ruta_carpeta, 'conjunto_de_datos')
        if os.path.isdir(ruta_conjunto_datos):
            # Buscar archivos .shp en 'conjunto_de_datos' y 'cdgv_1_250_000_serie1'
            for ruta in [ruta_conjunto_datos, os.path.join(ruta_conjunto_datos, 'cdgv_1_250_000_serie1')]:
                if os.path.isdir(ruta):
                    archivos_shp = buscar_archivos_shp(ruta)
                    if archivos_shp:
                        for archivo in archivos_shp:
                            print(f"Procesando archivo: {archivo}")
                            gdf = asignar_sistema_coordenadas(gpd.read_file(archivo), crs_original)
                            nombre_archivo = os.path.basename(archivo)
                            # Extraer la terminación del archivo
                            nombre_sin_extension = os.path.splitext(nombre_archivo)[0]  # Sin '.shp'
                            # Tomar la parte después del prefijo (supuesto prefijo de 5 caracteres)
                            partes = nombre_sin_extension[5:]  # Parte después del prefijo
                            # Obtener las posibles terminaciones
                            terminaciones_posibles = ['lg', 'pg', 'ptg']
                            terminacion = None
                            for pos in terminaciones_posibles:
                                if partes.endswith(pos):  # Verifica si termina con una de las terminaciones posibles
                                    terminacion = pos
                                    break
                            if terminacion and terminacion in grupos_shp:
                                grupos_shp[terminacion].append(gdf)
                            else:
                                print(f"Terminación no esperada: {terminacion}")

# Unir y reproyectar los GeoDataFrames por terminación
for terminacion, gdfs in grupos_shp.items():
    if gdfs:
        # Unir archivos con la misma terminación
        gdf_unido = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=crs_original)
        print(f"GeoDataFrame {terminacion} unido tiene {len(gdf_unido)} elementos")
        # Reproyectar al sistema de coordenadas final
        gdf_reproyectado = reproyectar_sistema_coordenadas(gdf_unido, crs_final)
        print(f"GeoDataFrame {terminacion} reproyectado tiene {len(gdf_reproyectado)} elementos")
        print(f"CRS del GeoDataFrame reproyectado {terminacion}: {gdf_reproyectado.crs}")
        
        # Asumiendo que 'zona_corte' ya está definida en tu código
        gdf_cortado = gpd.overlay(gdf_reproyectado, zona_corte, how='intersection')
        
        if not gdf_cortado.empty:
            nombre_salida = f"C{cvegeo}_{terminacion.upper()}_INEGI_SERIE_I.shp"
            ruta_salida = os.path.join(carpeta_zip, nombre_salida)
            gdf_cortado.to_file(ruta_salida)
            print(f"Archivo unido y cortado guardado en: {ruta_salida}")
        else:
            print(f"El GeoDataFrame resultante de {terminacion} está vacío después del corte.")

print("Proceso de unión y corte completo.")

Procesando archivo: CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202lg.shp
Procesando archivo: CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202pg.shp
Procesando archivo: CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202ptg.shp
Procesando archivo: CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202lg.shp
Procesando archivo: CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202pg.shp
Procesando archivo: CARTAS GEOLOGICAS\F12-2\conjunto_de_datos\cdgv_1_250_000_serie1\f1202ptg.shp
Procesando archivo: CARTAS GEOLOGICAS\F12-3\conjunto_de_datos\cdgv_1_250_000_serie1\f1203lg.shp
Procesando archivo: CARTAS GEOLOGICAS\F12-3\conjunto_de_datos\cdgv_1_250_000_serie1\f1203pg.shp
Procesando archivo: CARTAS GEOLOGICAS\F12-3\conjunto_de_datos\cdgv_1_250_000_serie1\f1203ptg.shp
Procesando archivo: CARTAS GEOLOGICAS\F12-3\conjunto_de_datos\cdgv_1_250_000_serie1\f1203lg.shp
Procesando archivo: CARTAS GEOLOGICAS